In [ ]:
!python ~/MasterProject/Code/ClinicaTools/clinica/clinicaml/cmdline.py iotools get-metadata ~/MasterProject/ADNI_data/ADNI_meta ~/MasterProject/DataAndExperiments/Data/DataPrep
!python ~/MasterProject/Code/ClinicaTools/clinica/clinicaml/cmdline.py iotools check-missing-modalities ~/MasterProject/DataAndExperiments/Data/BIDS ~/MasterProject/DataAndExperiments/Data/DataStat/missing_modes

In [ ]:
!python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py preprocessing quality-check t1-linear /u/horlavanasta/MasterProject/DataAndExperiments/Data/CAPS ~/MasterProject/DataAndExperiments/Data/DataStat/qc.tsv

In [1]:
!python ~/MasterProject/Code/ClinicaTools/clinica/clinicaml/cmdline.py iotools merge-tsv /u/horlavanasta/MasterProject/DataAndExperiments/Data/BIDS ~/MasterProject/DataAndExperiments/Data/DataPrep/metadata_merge.tsv ~/MasterProject/DataAndExperiments/Data/DataStat

[Warning] The PYTHONPATH environment variable is not empty. Make sure there is no interference with Clinica (content of PYTHONPATH: /mpcdf/soft/SLE_12/packages/skylake/pytorch/gpu-cuda-11.2/anaconda_3_2020.02-2020.02/1.8.1/lib/python3.7/site-packages).
End of BIDS information merge.


Manufacturer and preprocessing

In [4]:
def set_pubfig():
        import seaborn as sns
#         sns.set_context("paper", rc={"font.size":14,"axes.titlesize":22,"axes.labelsize":22})   
#         sns.axes_style("whitegrid")
        sns.set(font_scale = 1.75)
        sns.set_style('white')
        
def plot_scanner_bars(merged_df, MS):
    import numpy as np
    def show_values_on_bars(axs):
        import numpy as np
        def _show_on_single_plot(ax, factor=0.025):
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height() +ax.get_ylim()[-1]*factor
                value = '{}'.format(p.get_height())
                ax.text(_x, _y, value, ha="center")

        if isinstance(axs, np.ndarray):
            for idx, ax in np.ndenumerate(axs):
                _show_on_single_plot(ax)
        else:
            _show_on_single_plot(axs)
                
    import matplotlib.pyplot as plt
    import seaborn as sns
    fig, axes = plt.subplots(1, 4, figsize=((int(10 * 4), int(6 * 1))))
    set_pubfig()
    
    sns.countplot(data=merged_df, x="Manufacturer", order=merged_df["Manufacturer"].value_counts().index, ax=axes[0], palette="Set3")

    axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha="right")
    show_values_on_bars(axes[0])
    axes[0].set_ylim(bottom=0, top=220)

    
    for i, (manufacturer, manufacturer_df) in enumerate(merged_df.groupby("Manufacturer", as_index=False)):
        manufacturer_df=manufacturer_df.rename(columns = {'Mfg Model': manufacturer})
        sns.countplot(data=manufacturer_df, x=manufacturer, order=manufacturer_df[manufacturer].value_counts().index, ax=axes[i+1], palette= "Set2")
        axes[i+1].set_ylim(bottom=0, top=220)
        axes[i+1].set_xticklabels(axes[i+1].get_xticklabels(), rotation=45, ha="right")
        
        show_values_on_bars(axes[i+1])

    fig.patch.set_facecolor('white')
    os.makedirs("../../plots", exist_ok=True)
    plt.savefig(os.path.join("../../plots", "scanner_dist_%s.png"%MS), bbox_inches="tight")
    plt.close()

In [5]:
def violinplot_age(merged_df, MS=None):
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig=plt.figure(figsize=(8, 8))
    set_pubfig()
    sns.catplot(x="Diagnosis", y="Age",
                hue="Sex", col="MS",
                data=merged_df, kind="violin", split=True,
                scale="count",
#                 height=4, aspect=.7
               );
    
#     sns.violinplot(data=merged_df, x="Diagnosis", y= "Age",
#                  hue="Sex",palette= "Set3", ax=ax, cut=0, 
#                  order= ["CN", "AD"], 
#                    split=True,scale="count" )
#     ax.set_ylim(bottom=50, top=92)
    os.makedirs("../../plots", exist_ok=True)
    plt.savefig(os.path.join("../../plots", "age_dist_%s_violinplot.png"%MS), bbox_inches="tight")

    plt.close()

In [6]:
def violinplot_score(merged_df, metric="MMSE"):
    
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax=plt.subplots(1,1,figsize=(8, 8))
    set_pubfig()

    sns.violinplot(data=merged_df, x="Diagnosis", y=metric,
                palette= "Set3", hue="Sex",ax=ax, cut=0, 
                scale="count", 
                order= ["CN", "AD"], 
                   inner="stick", 
                   split=True,
                  )
    os.makedirs("../../plots", exist_ok=True)
    plt.savefig(os.path.join("../../plots", "%s_dist_violinplot.png"%(metric)), bbox_inches="tight")
#     plt.show()
    plt.close()
    
def histplot_score(merged_df, metric="MMSE"):
    
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax=plt.subplots(1,1,figsize=(8, 8))
    set_pubfig()

    sns.histplot(data=merged_df, x=metric, hue="Diagnosis",
                palette= "Set2", ax=ax, 
                stat="count", 
#                 order= ["CN", "AD"], 
#                    split=True,
                  )
    os.makedirs("../../plots", exist_ok=True)
    plt.savefig(os.path.join("../../plots", "%s_dist_histplot.png"%(metric)), bbox_inches="tight")
#     plt.show()
    plt.close()

def violinplot_weightTime(merged_df, metric="TE"):
    
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax=plt.subplots(1,1,figsize=(8, 8))
    set_pubfig()

    sns.violinplot(data=merged_df, x="MS", y=metric,
                palette= "Set3", hue="Diagnosis",ax=ax, cut=0, 
                scale="count", 
                hue_order= ["CN", "AD"], 
                inner="stick", 
                   split=True,)
    os.makedirs("../../plots", exist_ok=True)
    plt.savefig(os.path.join("../../plots", "%s_dist_violinplot.png"%(metric)), bbox_inches="tight")
#     plt.show()
    plt.close()
    


In [7]:
def boxplot_score(merged_df, metric="MMSE"):
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax=plt.subplots(1,1,figsize=(8, 8))
    set_pubfig()

    sns.boxplot(data=merged_df, x="MS", y=metric,
                palette= "Set3", hue="Diagnosis",ax=ax, 
                hue_order= ["CN", "AD"], 
               )
    os.makedirs("../../plots", exist_ok=True)
    plt.savefig(os.path.join("../../plots", "%s_dist_boxplot.png"%(metric)), bbox_inches="tight")
#     plt.show()
    plt.close()

def boxplot_weightTime(merged_df, metric="TE"):
    
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax=plt.subplots(1,1,figsize=(8, 8))
    set_pubfig()

    sns.boxplot(data=merged_df, x="MS", y=metric,
                palette= "Set3", hue="Diagnosis",ax=ax, 
                hue_order= ["CN", "AD"]
               )
    os.makedirs("../../plots", exist_ok=True)
    plt.savefig(os.path.join("../../plots", "%s_dist_boxplot.png"%(metric)), bbox_inches="tight")
#     plt.show()
    plt.close()

In [8]:
def get_merged_df(labels_files, metadata_file):
    import pandas as pd
    metadata_df=pd.read_csv(metadata_file, sep="\t")
    merged_dict={}
    for key in labels_files.keys():
        labels_info=pd.read_csv(labels_files[key], sep="\t")
        merged_dict[key]=metadata_df.merge(labels_info, on=["participant_id"], how="right")
    return pd.concat([merged_dict["AD"], merged_dict["CN"]], axis=0)


In [19]:
def print_data_stat(df, MS):
    import numpy as np
    print("Stat on %s"%MS)
    print(df.groupby("Diagnosis")["Sex"].value_counts())
    tmp_df=df[["Diagnosis", "Age"]].groupby("Diagnosis")
    print(tmp_df.agg([np.mean, np.std, np.min,np.max]))    
    tmp_df=df[["Diagnosis", "MMSE"]].groupby("Diagnosis")
    print(tmp_df.agg([np.mean, np.std, np.min,np.max]))  
    print("Site stat, \n", len(df.site.unique()))

    print("TI stat, [%f, %f]\n"%(df.TI.min(), df.TI.max()), df.TI.value_counts())
    print("TR stat, [%f, %f]\n"%(df.TR.min(), df.TR.max()), df.TR.value_counts())
    print("TE stat, [%f, %f]\n"%(df.TE.min(), df.TE.max()), df.TE.value_counts())
   
    
#     print("%f +- %f [%f, %f]"%(tmp_df].groupby("Diagnosis")["Sex"].mean(), df.Age.std(), df.Age.min(), df.Age.max()))    
    print("\n_______________________________________________________________________________ \n ")

In [20]:
def rename_cols(df):
    return df.rename(columns = {'diagnosis_y': "Diagnosis", 'sex_y': "Sex","age_bl":"Age", "cdr_sb":"CDR"})


In [21]:
import pandas as pd
df=pd.DataFrame()
MS="1.5T-3T"
metadata_file="/u/horlavanasta/MasterProject/DataAndExperiments/Data/DataStat/merge.tsv"
labels_files={"AD":"/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-%s/labels/AD.tsv"%MS, 
          "CN":"/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-%s/labels/CN.tsv"%MS}
MS_df=rename_cols(get_merged_df(labels_files, metadata_file))

print_data_stat(MS_df, MS)
# violinplot_age(MS_df, MS)
for MS in ["1.5T", "3T"]:
    metadata_file="/u/horlavanasta/MasterProject/DataAndExperiments/Data/DataStat/merge.tsv"
    labels_files={"AD":"/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-%s/labels/AD.tsv"%MS, 
              "CN":"/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-%s/labels/CN.tsv"%MS}
    MS_df=rename_cols(get_merged_df(labels_files, metadata_file))
    MS_df=MS_df.assign(MS=MS)
    df=pd.concat([df, MS_df], axis=0)
#     violinplot_age(MS_df, MS)
#     plot_scanner_bars(MS_df, MS)

#     print_data_stat(MS_df, MS)

violinplot_age(df, "both")

# violinplot_score(df)
# violinplot_score(df, metric="FAQ") 
# histplot_score(df)
# histplot_score(df, metric="FAQ") 
# histplot_score(df, metric="CDR") 
# histplot_score(df, metric="TE") 
# histplot_score(df, metric="TR") 
# histplot_score(df, metric="TI") 

# violinplot_weightTime(df,metric="TE")
# violinplot_weightTime(df,metric="TR")
# violinplot_weightTime(df,metric="TI")
# # boxplot_score(df,metric="MMSE")  
# boxplot_score(df,metric="CDR")
# boxplot_weightTime(df,metric="TE")
# boxplot_weightTime(df,metric="TR")

Stat on 1.5T-3T
Diagnosis  Sex
AD         M      147
           F      124
CN         M      158
           F      154
Name: Sex, dtype: int64
                 Age                      
                mean       std  amin  amax
Diagnosis                                 
AD         75.032103  7.688387  55.1  90.9
CN         75.032372  5.399000  59.8  89.6
                MMSE                      
                mean       std  amin  amax
Diagnosis                                 
AD         23.265683  2.203424  14.0  28.0
CN         29.080128  1.077386  25.0  30.0
Site stat, 
 60
TI stat, [0.000000, 1043.000000]
 1000.0    367
0.0        84
900.0      81
400.0      50
1043.0      1
Name: TI, dtype: int64
TR stat, [6.764800, 3000.000000]
 3000.0000    110
2300.0000     81
2400.0000     58
8.9160        34
8.5920        32
            ... 
11.0400        1
6.8072         1
8.6041         1
8.6175         1
8.5075         1
Name: TR, Length: 73, dtype: int64
TE stat, [2.848000, 4.960000

<Figure size 576x576 with 0 Axes>

In [ ]:
import pandas as pd
import seaborn as sns
metadata_file="/u/horlavanasta/MasterProject/DataAndExperiments/Data/DataStat/merge.tsv"
labels_files={"AD":"/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T-3T/labels/AD.tsv", 
              "CN":"/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T-3T/labels/CN.tsv"}
unite_diagnosis=True
metadata_df=pd.read_csv(metadata_file, sep="\t")
merged_dict={}
for key in labels_files.keys():
    labels_info=pd.read_csv(labels_files[key], sep="\t")
    merged_dict[key]=metadata_df.merge(labels_info, on=["participant_id"], how="right")
    
if unite_diagnosis: 
    merged_df=pd.concat([merged_dict["AD"], merged_dict["CN"]], axis=0)
    
# plot_scanner_bars(merged_df, "1.5T-3T")
# plot_preprocessing_pie(merged_df, "1.5T-3T")
print(merged_df.original_study.value_counts())
print(merged_df.visitIdentifier.value_counts())

# print(merged_df.groupby("Manufacturer")["Mfg Model"].value_counts())
# print(merged_df.preprocessing.value_counts())

    

In [ ]:
import pandas as pd
metadata_file="/u/horlavanasta/MasterProject/DataAndExperiments/Data/DataStat/merge.tsv"
labels_files={"AD":"/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T/labels/AD.tsv", 
              "CN":"/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T/labels/CN.tsv"}
unite_diagnosis=True
metadata_df=pd.read_csv(metadata_file, sep="\t")
merged_dict={}
for key in labels_files.keys():
    labels_info=pd.read_csv(labels_files[key], sep="\t")
    merged_dict[key]=metadata_df.merge(labels_info, on=["participant_id"], how="right")
    
if unite_diagnosis: 
    merged_df=pd.concat([merged_dict["AD"], merged_dict["CN"]], axis=0)
    
plot_scanner_bars(merged_df, "1.5T")

# print(merged_df.Manufacturer.value_counts())
# print(merged_df.groupby("Manufacturer")["Mfg Model"].value_counts())
print(merged_df.preprocessing.value_counts())


In [ ]:
import pandas as pd
metadata_file="/u/horlavanasta/MasterProject/DataAndExperiments/Data/DataStat/merge.tsv"
labels_files={"AD":"/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-3T/labels/AD.tsv", 
              "CN":"/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-3T/labels/CN.tsv"}
unite_diagnosis=True
metadata_df=pd.read_csv(metadata_file, sep="\t")
merged_dict={}
for key in labels_files.keys():
    labels_info=pd.read_csv(labels_files[key], sep="\t")
    merged_dict[key]=metadata_df.merge(labels_info, on=["participant_id"], how="right")
    
if unite_diagnosis: 
    merged_df=pd.concat([merged_dict["AD"], merged_dict["CN"]], axis=0)

plot_scanner_bars(merged_df, "3T")
# print(merged_df.Manufacturer.value_counts())
# print(merged_df.groupby("Manufacturer")["Mfg Model"].value_counts())
print(merged_df.preprocessing.value_counts())


In [ ]:
for MS in ["3T", "1.5T"]:
    metadata_file="/u/horlavanasta/MasterProject/DataAndExperiments/Data/DataStat/merge.tsv"
    labels_files={"AD":"/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-%s/labels/AD.tsv"%MS, 
              "CN":"/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-%s/labels/CN.tsv"%MS}
    df=get_merged_df(labels_files, metadata_file)
    violinplot_age(df, MS)